# California Housing Challenge

The notebook is intended to predict the average house value upon the provided house features.

In [ ]:
# Import Standard Libraries
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Read Data

In [ ]:
# Read training data
california_housing_train = pd.read_csv('./../../data/season_3_episode_1/california_housing_train.csv')

In [ ]:
california_housing_train.head()

In [ ]:
california_housing_train.info()

# Exploratory Data Analysis (EDA)

## Median Income Distribution

In [ ]:
# Plot the distribution of the column 'MedInc'
ax = sns.histplot(data=california_housing_train, 
                  x='MedInc')

ax.set_title('Median Income Distribution')

plt.show()

The data have a binomial distribution.

## Median House Value per Median Income